In [1]:
import pandas as pd
import numpy as np
import keras
from keras.layers import Embedding, Input, LSTM, Dense, Subtract
from keras.models import Model
from keras.preprocessing import text
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [6]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data.fillna("0", inplace=True)
test_data.fillna("0", inplace=True)
num_samples_train = len(train_data)
num_samples_test = len(test_data)
print("Total number of train samples: %s" % num_samples_train)
print("Total number of test samples: %s" % num_samples_test)

Total number of train samples: 404290
Total number of test samples: 2345796


In [7]:
ques1_train = train_data['question1'].tolist()
ques2_train = train_data['question2'].tolist()
labels_train = train_data['is_duplicate']

ques1_test = test_data['question1'].tolist()
ques2_test = test_data['question2'].tolist()

In [8]:
texts = ques1_train + ques2_train + ques1_test + ques2_test
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print('Number of sentences tokenizer was trained on: %s' % tokenizer.document_count)

Found 137042 unique tokens.
Number of sentences tokenizer was trained on: 5500172


In [9]:
max_seq_len = 15
data = pad_sequences(sequences, maxlen=max_seq_len)
ques1_train = data[:num_samples_train]
ques2_train = data[num_samples_train: 2*num_samples_train]
ques1_test = data[2* num_samples_train: 2*num_samples_train + num_samples_test]
ques2_test = data[2*num_samples_train + num_samples_test:]

In [10]:
num_samples_train = len(train_data)
num_samples_test = len(test_data)

In [11]:
embeddings_index = {}
f = open('glove.42B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))


Found 1917494 word vectors.


In [12]:
EMBEDDING_DIM = 300
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1, 
                            EMBEDDING_DIM, 
                            weights=[embedding_matrix], 
                            input_length=max_seq_len, 
                            trainable=False)

In [13]:
ques1_input = Input(shape=(max_seq_len,), dtype='int32')
ques1_embedded = embedding_layer(ques1_input)
ques1_output = LSTM(50)(ques1_embedded)

ques2_input = Input(shape=(max_seq_len,), dtype='int32')
ques2_embedded = embedding_layer(ques2_input)
ques2_output = LSTM(50)(ques2_embedded)

difference_tensor = Subtract()([ques1_output, ques2_output])
predictions = Dense(1, activation='sigmoid')(difference_tensor)

model = Model(inputs=[ques1_input, ques2_input], outputs=predictions)

In [14]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy')
model.fit([ques1_train, ques2_train], np.array(labels_train), epochs=20)

Epoch 1/20
404290/404290 [==============================] - 292s 723us/step - loss: 0.5251
Epoch 2/20
404290/404290 [==============================] - 284s 703us/step - loss: 0.4755
Epoch 3/20
404290/404290 [==============================] - 281s 694us/step - loss: 0.4449
Epoch 4/20
404290/404290 [==============================] - 281s 695us/step - loss: 0.4192
Epoch 5/20
404290/404290 [==============================] - 280s 693us/step - loss: 0.3972
Epoch 6/20
404290/404290 [==============================] - 281s 694us/step - loss: 0.3771
Epoch 7/20
404290/404290 [==============================] - 282s 697us/step - loss: 0.3596
Epoch 8/20
404290/404290 [==============================] - 283s 699us/step - loss: 0.3433
Epoch 9/20
404290/404290 [==============================] - 283s 699us/step - loss: 0.3289
Epoch 10/20
404290/404290 [==============================] - 282s 698us/step - loss: 0.3157
Epoch 11/20
404290/404290 [==============================] - 283s 700us/step - loss: 0.30

In [15]:
preds_test = model.predict([ques1_test, ques2_test])

In [16]:
result_df = pd.DataFrame({'test_id': test_data['test_id'], 'is_duplicate':preds_test[:,0]}, 
                         columns=['test_id', 'is_duplicate'])
result_df.to_csv('submissions.csv', header=['test_id', 'is_duplicate'], index=False)